# Lab 2 - CMC-13
Equipe:
* Emanuel Reinaldo Gomes Bezerra
* Pedro Pinheiro Borges
* Guilherme Alt Chagas Merklein

### 1. Preparação dos dados

In [ ]:
# code

### 2. Criação dos três modelos

#### 2.a) Modelo baseado em kNN, Árvore de Decisão ou SVM

In [ ]:
# code

#### 2.b) Modelo baseado em Redes Neurais do tipo MLP (MultiLayer Perceptron)

In [ ]:
# code

#### 2.c) Modelo baseado em Comitês (Random Forests, AdamBoost, etc)

In [ ]:
# code

### 3. Análise comparativa do desempenho dos modelos

In [ ]:
# code

### 4. Aplicação da predição do Modelo Desenvolvido

In [ ]:
# code

### 5. Conclusões